In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# basic import
import os
import os.path as op
import sys
sys.path.insert(0, op.dirname(os.getcwd()))

# python libs
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [18, 8]

# tk libs
from lib.objs.tkpaths import Site
from lib.data_fetcher.STORMS import Download_NOAA_WMO
from lib.tcyclone import Extract_Circle
from lib.plotting.storms import WorldMap_Storms
from lib.statistical import CopulaSimulation


# --------------------------------------
# Site paths and parameters
site = Site('KWAJALEIN')

DB = site.pc.DB                        # common database
ST = site.pc.site                      # site database
PR = site.params                       # site parameters

# input files
p_hist_r1_params = ST.TCs.hist_r1_params
p_hist_r2_params = ST.TCs.hist_r2_params

# output files


# parameters
num_sim_rnd = int(PR.TCS.num_simulate_rnd)


In [2]:
# --------------------------------------
# Probabilistic simulation Historical TCs

# aux function
def FixPareto(var):
    'Fix data. It needs to start at 0 for Pareto adjustment '
    
    mx = np.amax(var)
    aux = mx + np.absolute(var - mx)
    var_pareto = aux - np.amin(aux) + 0.00001
    
    return var_pareto


# use small radius (4º)
xds_TCs_r2_params = xr.open_dataset(p_hist_r2_params)

pmean = xds_TCs_r2_params.pressure_mean.values[:]
pmin = xds_TCs_r2_params.pressure_min.values[:]
gamma = xds_TCs_r2_params.gamma.values[:]
delta = xds_TCs_r2_params.delta.values[:]
vmean = xds_TCs_r2_params.velocity_mean.values[:]

# fix pressure for pareto adjustment (start at 0)
pmean_p = FixPareto(pmean)
pmin_p = FixPareto(pmin)

# join storm parameters for copula simulation
storm_params = np.column_stack(
    (pmean_p, pmin_p, gamma, delta, vmean)
)
    
# statistical simulate PCs using copulas 
kernels = ['GPareto', 'GPareto', 'ECDF', 'ECDF', 'ECDF']
storm_params_sim = CopulaSimulation(storm_params, kernels, num_sim_rnd)

print(storm_params_sim)


/Users/nico/Projects/TESLA-kit/source/venv3/lib/python3.7/site-packages/scipy/stats/_continuous_distns.py:2247: RuntimeWarning: divide by zero encountered in true_divide
  val = val + cnk * (-1) ** ki / (1.0 - c * ki)
/Users/nico/Projects/TESLA-kit/source/venv3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1035: RuntimeWarning: invalid value encountered in subtract
  mu2 = mu2p - mu * mu
/Users/nico/Projects/TESLA-kit/source/venv3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2316: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu


[[ 15.39983142  21.13264496  95.42802283 120.99241645  33.50607342]
 [  7.23927484  10.27641394 132.08652886 108.51463555  15.20414207]
 [  3.43518583   7.75157125 103.03390751 137.02694902  18.00245675]
 ...
 [  5.07536586   9.23427729  84.30796711  80.89582397  10.40852508]
 [ 11.98048369  18.60185825  94.5472342   74.99942097  20.5322163 ]
 [ 18.80530358  18.15562648 187.8475076  170.21566482  16.26791979]]
